In [11]:
!pip install sentence_transformers
!pip install bitsandbytes

# 1) Data Loading & Pre-processing

In [12]:
import pandas as pd
df = pd.read_excel("/kaggle/input/datsasetnlp/Main.xlsx")

In [13]:
df.head()

,Batch,Source Language,ID,Type,Text,English MT,Arabic MT,Annotator ID,Bias,Propaganda,Type of Propaganda,Type of Bias,Comments
0,B01,English,1,MAIN,Yemen's Houthis have waded into the Israel-Ham...,Yemen's Houthis have waded into the Israel-Ham...,خاض الحوثيون في اليمن الحرب بين إسرائيل وحماس ...,1.0,Biased against Palestine,Not Propaganda,Propaganda Not to be deleted,ضمني,NaN
1,B01,English,2,MAIN,Isreal - Hamas Conflict | Face to Face,Isreal - Hamas Conflict | Face to Face,إسرائيل - الصراع مع حماس | وجها لوجه,4.0,Unbiased,Not Propaganda,Not Propaganda,NaN,NaN
2,B01,English,4,MAIN,Videos show how armed men from Gaza stormed a ...,Videos show how armed men from Gaza stormed a ...,أظهرت مقاطع فيديو كيف اقتحم مسلحون من غزة مهرج...,3.0,Unbiased,Not Propaganda,Propaganda Not to be deleted,NaN,NaN
3,B01,English,7,MAIN,Protest in Aligarh Muslim University in suppor...,Protest in Aligarh Muslim University in suppor...,وقفة احتجاجية في جامعة عليكرة الإسلامية دعما ل...,5.0,Unbiased,Not Propaganda,Not Propaganda,NaN,NaN
4,B01,English,8,MAIN,IDF releases audio recording about misfired ro...,IDF releases audio recording about misfired ro...,الجيش الإسرائيلي ينشر تسجيلًا صوتيًا حول صاروخ...,2.0,Biased against Palestine,Propaganda,Propaganda Must be deleted,ضمني,NaN


In [14]:
df.columns

Index(['Batch', 'Source Language', 'ID', 'Type', 'Text', 'English MT',
       'Arabic MT', 'Annotator ID', 'Bias', 'Propaganda', 'Type of Propaganda',
       'Type of Bias', 'Comments'],
      dtype='object')

In [15]:
df = df.drop(columns=['Annotator ID', 'Text','Batch', 'ID', 'Comments', 'Type', 'Source Language', 'English MT',"Type of Bias","Propaganda","Type of Propaganda"], errors='ignore')

In [16]:
df.head()

,Arabic MT,Bias
0,خاض الحوثيون في اليمن الحرب بين إسرائيل وحماس ...,Biased against Palestine
1,إسرائيل - الصراع مع حماس | وجها لوجه,Unbiased
2,أظهرت مقاطع فيديو كيف اقتحم مسلحون من غزة مهرج...,Unbiased
3,وقفة احتجاجية في جامعة عليكرة الإسلامية دعما ل...,Unbiased
4,الجيش الإسرائيلي ينشر تسجيلًا صوتيًا حول صاروخ...,Biased against Palestine


In [17]:
total_rows = len(df)
print(f"Total number of rows: {total_rows}")

Total number of rows: 13500


In [18]:
df.isna().sum()

Arabic MT       0
Bias         2700
dtype: int64

In [19]:
import numpy as np

In [20]:
df.replace(['', 'None', 'nan'], np.nan, inplace=True)  
df.dropna(subset=['Bias'], inplace=True)

In [21]:
df.isna().sum()

Arabic MT    0
Bias         0
dtype: int64

In [22]:
df.head()

,Arabic MT,Bias
0,خاض الحوثيون في اليمن الحرب بين إسرائيل وحماس ...,Biased against Palestine
1,إسرائيل - الصراع مع حماس | وجها لوجه,Unbiased
2,أظهرت مقاطع فيديو كيف اقتحم مسلحون من غزة مهرج...,Unbiased
3,وقفة احتجاجية في جامعة عليكرة الإسلامية دعما ل...,Unbiased
4,الجيش الإسرائيلي ينشر تسجيلًا صوتيًا حول صاروخ...,Biased against Palestine


In [23]:
total_rows = len(df)
print(f"Total number of rows: {total_rows}")

Total number of rows: 10800


In [24]:
print(df['Bias'].cat.categories if df['Bias'].dtype.name == 'category' else df['Bias'].unique())

['Biased against Palestine' 'Unbiased' 'Unclear' 'Biased against others'
 'Biased against Israel' 'Biased against both Palestine and Israel'
 'Not Applicable']


In [25]:
columns_to_check = ['Bias']

for column in columns_to_check:
    print(f"Class counts for '{column}':")
    print(df[column].value_counts())
    print("\n")

Class counts for 'Bias':
Bias
Unbiased                                    6817
Biased against Palestine                    2900
Unclear                                      432
Biased against Israel                        281
Biased against others                        203
Not Applicable                               120
Biased against both Palestine and Israel      47
Name: count, dtype: int64




In [26]:
!pip install emoji

In [27]:
import re
import emoji

def preprocess_text(text):
    # Ensure text is a string
    text = str(text)
    
    # 1. Remove hashtags
    text = re.sub(r'#\w+', '', text)
    
    # 2. Remove full URLs with protocols (e.g., "http://example.com")
    text = re.sub(r'https?://\S+|www\.\S+|bit\.ly\S*', '', text)
    
    # 3. Remove standalone paths without protocols (e.g., "/content/kan-news/defense/629514/")
    text = re.sub(r'/\S+', '', text)
    
    # 4. Remove emails
    text = re.sub(r'\S+@\S+', '', text)
    
    # 5. Remove emojis
    text = emoji.replace_emoji(text, replace='')
    
    # 6. Remove diacritics
    arabic_diacritics = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(arabic_diacritics, '', text)
    
    # 7. Remove Tatweel (ـ)
    text = re.sub(r'ـ', '', text)
    
    # 8. Normalize Arabic text
    text = re.sub(r'[إأآا]', 'ا', text)  # Unify Alif variants
    text = re.sub(r'ة', 'ه', text)  # Replace Taa Marbuta with Haa
    text = re.sub(r'ى', 'ي', text)  # Replace Alef Maqsura with Ya
    
    # 9. Remove repeated characters (e.g., "ممتتتاز" → "ممتاز")
    text = re.sub(r'(.)\1+', r'\1', text)
    
    # Return cleaned text
    return text

In [28]:
# Apply the preprocessing function
df['Arabic MT'] = df['Arabic MT'].apply(preprocess_text)

In [29]:
df['Arabic MT'][9022]

'اولا علي قناه ABC: جلست المرشحه الرئاسيه الجمهوريه نيكي هيلي مع لينسي ديفيس من قناه ABC News، حيث ناقشت مجموعه واسعه من المواضيع بما في ذلك الرئيس السابق. ترامب والحرب بين اسرائيل وحماس والاجهاض وحياتها قبل ان تظهر امام الجمهور. اقرا المزيد: '

# Handling imbalanced dataset

In [30]:
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import BorderlineSMOTE
import numpy as np

model = SentenceTransformer("intfloat/multilingual-e5-large")

X = df['Arabic MT']
y = df['Bias']

X_embeddings = model.encode(X.tolist(), convert_to_numpy=True)

X_train, X_test, y_train, y_test = train_test_split(X_embeddings, y, test_size=0.2, stratify=y, random_state=42)


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Batches:   0%|          | 0/338 [00:00<?, ?it/s]

In [31]:
from imblearn.over_sampling import BorderlineSMOTE

borderline_smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = borderline_smote.fit_resample(X_train, y_train)

print("Class distribution after Borderline-SMOTE:")
print(pd.Series(y_train_resampled).value_counts())

Class distribution after Borderline-SMOTE:
Bias
Unbiased                                    5454
Biased against Palestine                    5454
Unclear                                     5454
Biased against Israel                       5454
Biased against others                       5454
Biased against both Palestine and Israel    5454
Not Applicable                              5454
Name: count, dtype: int64


In [32]:
from sklearn.metrics.pairwise import cosine_similarity

original_embeddings = X_train
synthetic_embeddings = X_train_resampled[len(X_train):]

cos_sim = cosine_similarity(original_embeddings, synthetic_embeddings)
average_similarity = np.mean(np.max(cos_sim, axis=0))
print(f"Average Cosine Similarity of Synthetic Samples: {average_similarity:.4f}")

if average_similarity < 0.7:
    print("Warning: Synthetic samples are significantly different from the original data.")
else:
    print("Synthetic samples appear similar to the original data.")

Average Cosine Similarity of Synthetic Samples: 0.9886
Synthetic samples appear similar to the original data.


In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

X_train_final, X_val, y_train_final, y_val = train_test_split(X_train_resampled, y_train_resampled, test_size=0.2, random_state=42)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_final, y_train_final)

y_pred = rf_model.predict(X_val)

print("Classification Report:")
print(classification_report(y_val, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))


Classification Report:
                                          precision    recall  f1-score   support

                   Biased against Israel       1.00      0.98      0.99      1082
                Biased against Palestine       0.79      0.81      0.80      1108
Biased against both Palestine and Israel       1.00      1.00      1.00      1099
                   Biased against others       1.00      0.99      0.99      1078
                          Not Applicable       0.99      0.99      0.99      1079
                                Unbiased       0.77      0.79      0.78      1107
                                 Unclear       0.99      0.97      0.98      1083

                                accuracy                           0.93      7636
                               macro avg       0.93      0.93      0.93      7636
                            weighted avg       0.93      0.93      0.93      7636

Confusion Matrix:
[[1065    0    0    0    0   17    0]
 [   0  903    1

In [64]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix

# Split the training data into training and validation sets
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train_resampled, y_train_resampled, test_size=0.2, random_state=42)

# Initialize and train the Naive Bayes model
nb_model = GaussianNB()
nb_model.fit(X_train_final, y_train_final)

# Make predictions on the validation set
y_pred = nb_model.predict(X_val)

# Print the classification report and confusion matrix
print("Classification Report:")
print(classification_report(y_val, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))


Classification Report:
                                          precision    recall  f1-score   support

                   Biased against Israel       0.74      0.73      0.73       702
                Biased against Palestine       0.39      0.55      0.46       700
Biased against both Palestine and Israel       0.99      0.92      0.95       701
                   Biased against others       0.73      0.77      0.75       674
                          Not Applicable       0.88      0.74      0.80       663
                                Unbiased       0.56      0.68      0.62       691
                                 Unclear       0.69      0.40      0.51       756

                                accuracy                           0.68      4887
                               macro avg       0.71      0.68      0.69      4887
                            weighted avg       0.71      0.68      0.69      4887

Confusion Matrix:
[[509  96   0  21   9  50  17]
 [ 59 388   2  42   2 1

In [50]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train_final.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(1))  # Output layer

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [51]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [53]:
X_train_resampled, X_val, y_train_resampled, y_val = train_test_split(X_train_resampled, y_train_resampled, test_size=0.2, random_state=42)

# Now reshape as needed for LSTM
X_train_final = X_train_resampled.reshape((X_train_resampled.shape[0], 1, X_train_resampled.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))


In [56]:
from sklearn.preprocessing import OneHotEncoder

# Ensure y_train_resampled is a NumPy array before reshaping
y_train_resampled = y_train_resampled.to_numpy()

# Now reshape the array
y_train_final = one_hot_encoder.fit_transform(y_train_resampled.reshape(-1, 1))

# Apply the same transformation to y_val
y_val = y_val.to_numpy()
y_val_onehot = one_hot_encoder.transform(y_val.reshape(-1, 1))

# Print the shapes to verify
print("y_train_final shape:", y_train_final.shape)
print("y_val_onehot shape:", y_val_onehot.shape)


y_train_final shape: (24433, 7)
y_val_onehot shape: (6109, 7)


/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  "The following categories were supposed to be "


In [58]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, message=".*sparse.*")


In [62]:
def create_lstm_model(optimizer='adam', dropout_rate=0.2):
    num_classes = y_train_final.shape[1]  # This should be the number of unique classes in the target labels
    
    model = Sequential([
        LSTM(64, input_shape=(X_train_final.shape[1], X_train_final.shape[2]), return_sequences=False),
        Dropout(dropout_rate),
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax')  # Adjust number of classes dynamically
    ])
    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [63]:
model = create_lstm_model()

# Train the LSTM model
model.fit(X_train_final, y_train_final, epochs=10, batch_size=32, validation_data=(X_val, y_val_onehot))

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val_onehot)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

Epoch 1/10
764/764 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.4621 - loss: 1.3756 - val_accuracy: 0.7289 - val_loss: 0.6825
Epoch 2/10
764/764 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7489 - loss: 0.6476 - val_accuracy: 0.7730 - val_loss: 0.5800
Epoch 3/10
764/764 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7921 - loss: 0.5393 - val_accuracy: 0.7946 - val_loss: 0.5377
Epoch 4/10
764/764 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8133 - loss: 0.4850 - val_accuracy: 0.8047 - val_loss: 0.4934
Epoch 5/10
764/764 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8264 - loss: 0.4505 - val_accuracy: 0.8199 - val_loss: 0.4526
Epoch 6/10
764/764 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8333 - loss: 0.4364 - val_accuracy: 0.8303 - val_loss: 0.4410
Epoch 7/10
764/764 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8454 - loss: 0.4045 - val_accuracy: 0.8317 - val_loss: 0.4434
Epoch 8/10
764/764 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8523 - loss: 0.3935 - val_accuracy: 0.